In [ ]:
# %pip install pandas
# %pip install facebook_scraper
# %pip install tqdm

In [ ]:
from facebook_scraper import get_posts
import pandas as pd
from tqdm import tqdm

In [ ]:
sent = "neutral"

In [ ]:

login = ['defow62637@mliok.com']
password = "PasswordLogin"

# create dataframe to store posts
data_frame = pd.DataFrame()

# Get posts from Moms Demand Action -- Clearly this is a gun control group

if sent == 'pro':
    pro_gun_control_posts = get_posts(
                account = 'guncontrolnow.campaign',
                pages=1000,
                page_limit = None,
                credentials=(login[0], password),
                #cookies='cookies.json',
                options={"progress": True, "posts_per_page": 200, "allow_extra_requests": False})
elif sent == 'against':
    anti_gun_control_posts = get_posts(
                account = 'GunOwners',
                pages=1000,
                page_limit = None,
                credentials=(login[0], password),
                #cookies='cookies.json',
                options={"progress": True, "posts_per_page": 200, "allow_extra_requests": False})
else:
    neutral_gun_control_posts = get_posts(
                post_urls = ['pfbid0ADZPihZN3bKp2yPZnbQHvkBkF6zWuWFq1CEvuoY25Nnw8QeLbHaiHSU5WSCcCuTCl'],
                credentials=(login[0], password),
                #cookies='cookies.json',
                options={"progress": True, "posts_per_page": 2000, "allow_extra_requests": True, 'comments': 1000})

posts = []
comments = []
dict = {}

def get_facebook_posts(sentiment):
    if sentiment == 'pro':
        for index, post in enumerate(pro_gun_control_posts):
            if index % 100 == 0:
                print(f'Post Number {index}')
                print(post['text'][:250])
            posts.append(post)

    elif sentiment == 'neutral':
        for index, post in enumerate(neutral_gun_control_posts):
            if index % 100 == 0:
                print(f'Post Number {index}')
                print(post['text'][:250])
                print(post['post_id'])

            for i, comment in enumerate(post['comments_full']):
                comments.append(comment)
            posts.append(post)
            
    else:
        for index, post in enumerate(anti_gun_control_posts):
            if index % 100 == 0:
                print(f'Post Number {index}')
                print(post['text'][:250])
            posts.append(post)
           






In [ ]:
def save_posts_list(df, sentiment):
    if sentiment != 'neutral':
        df.to_csv(f'data/{sentiment}1-posts.csv')

# If the data has already been scraped and collected -- Load the Data

In [ ]:
get_facebook_posts(sent) # get pro gun control posts 


In [ ]:
data_frame = pd.DataFrame(posts)
data_frame = data_frame[['post_id', 'time', 'text', 'reaction_count']]
if sent == 'pro':
    data_frame['label'] = 1
elif sent == 'neutral':
    data_frame['label'] = -1
else:
    data_frame['label'] = 0


In [ ]:
import re
def text_preprocessing(text):
    # Remove entity mentions starting with '@user'
    text = re.sub(r'@\w+', '', text)
    
    # Remove URLs
    text = re.sub(r'(http|https)://[^\s]+', '', text)
    
    emojis_pattern = re.compile("[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]|[\U00002600-\U000027BF]|[\U0001F900-\U0001F9FF]|[\U0001F170-\U0001F251]|[\U0001F300-\U0001F5FF]|[\U00002702-\U000027B0]|[\U0001F600-\U0001F636]|[\U000024C2-\U0001F251]|[\U0001F30D-\U0001F567]", flags=re.UNICODE)
    text = emojis_pattern.sub('', text)
    
    # Correct common errors
    text = text.replace('&amp;', '&')
    
    # Remove excess whitespace characters
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove Paragraph Spacing
    text = text.replace('\n', ' ')
    
    return text

# Output to CSV file after preprocessing 

In [ ]:
# Preprocess text

data_frame['text'] = data_frame['text'].apply(lambda x: text_preprocessing(x))

# Remove empty posts
data_frame = data_frame[data_frame['text'] != '']

# Remove duplicate posts
data_frame = data_frame.drop_duplicates(subset='text')

# Remove posts with less than 10 characters
data_frame = data_frame[data_frame['text'].str.len() >= 10]

# Save posts
save_posts_list(data_frame, sent)


# Working with Comments 

In [ ]:
if len(comments) != 0:
    comment_dataframe = pd.DataFrame.from_dict(comments)
    comment_dataframe = comment_dataframe[['comment_id', 'comment_text', 'comment_time']]
    comment_dataframe.set_index('comment_time', inplace=True)
    comment_dataframe.to_csv('comment/comments.csv')



    
